In [1]:
import pandas as pd
import pickle

print("--- Budowanie mapy synonimów dla dzielnic ---")

# Wczytujemy plik z lokalizacjami
df_lokalizacja_raw = pd.read_csv('lokalizacja.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False)
df_lokalizacja_raw.columns = ['LocationID', 'ParentId', 'Name', 'Type', 'Path']
df_lokalizacja = df_lokalizacja_raw.dropna(subset=['LocationID', 'Name', 'Type', 'ParentId']).copy()

# Konwersja ID na liczby
for col in ['LocationID', 'ParentId']:
    df_lokalizacja[col] = pd.to_numeric(df_lokalizacja[col], errors='coerce').fillna(0).astype(int)

# Wyodrębniamy dzielnice i osiedla (części miasta)
df_dzielnice = df_lokalizacja[df_lokalizacja['Type'] == 'Dzielnica'][['LocationID', 'Name']].rename(columns={'Name': 'Dzielnica_Name'})
df_osiedla = df_lokalizacja[df_lokalizacja['Type'] == 'Część miasta'][['ParentId', 'Name']].rename(columns={'ParentId': 'LocationID', 'Name': 'Osiedle_Name'})

# Łączymy osiedla z nazwami ich nadrzędnych dzielnic
df_mapa = pd.merge(df_osiedla, df_dzielnice, on='LocationID', how='inner')

# Tworzymy słownik: nazwa_osiedla -> nazwa_dzielnicy
synonimy_map = pd.Series(df_mapa.Dzielnica_Name.values, index=df_mapa.Osiedle_Name.str.lower()).to_dict()

# Zapisujemy słownik do pliku
with open('synonimy_dzielnic.pkl', 'wb') as f:
    pickle.dump(synonimy_map, f)

print(f"Mapa synonimów została zbudowana i zapisana do 'synonimy_dzielnic.pkl'.")
print(f"Znaleziono {len(synonimy_map)} mapowań.")
print("\nPrzykład: Saska Kępa ->", synonimy_map.get('saska kępa'))

--- Budowanie mapy synonimów dla dzielnic ---
Mapa synonimów została zbudowana i zapisana do 'synonimy_dzielnic.pkl'.
Znaleziono 210 mapowań.

Przykład: Saska Kępa -> Praga-południe
